In [ ]:
import os
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(filename='deleted_qc_raw_files.log', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def delete_qc_raw_files(directory, exclude_most_recent=False, exclude_folder_prefix=None):
    """
    Walks through all subdirectories of the provided directory and deletes files 
    with a `.raw` extension and 'QC' appearing at positions 9 and 10 in the filename.
    Optionally excludes the most recent folder and any folder with a specific prefix.

    Args:
        directory (str): The root directory to search for files.
        exclude_most_recent (bool): Whether to exclude the most recent folder from the search.
        exclude_folder_prefix (str): A string representing a folder name (or part of it) 
                                     to exclude from deletion (e.g., '202407').

    Logs:
        INFO: Logs each successfully deleted file with its full path.
        ERROR: Logs errors encountered during file deletion with the error message.

    Returns:
        None
    """
    most_recent_folder = None

    if exclude_most_recent:
        # Identify the most recent folder by modification time
        most_recent_folder = max(
            (os.path.join(directory, subfolder) for subfolder in os.listdir(directory)
             if os.path.isdir(os.path.join(directory, subfolder))),
            key=os.path.getmtime,
            default=None
        )
        logging.info(f"Most recent folder (excluded): {most_recent_folder}")

    # Walk through all directories and subdirectories
    for foldername, subfolders, filenames in os.walk(directory):
        # Skip the most recent folder if exclude_most_recent is True
        if exclude_most_recent and foldername == most_recent_folder:
            logging.info(f"Skipping folder (most recent): {foldername}")
            continue

        # Skip the folder if it starts with the exclude_folder_prefix
        if exclude_folder_prefix and os.path.basename(foldername).startswith(exclude_folder_prefix):
            logging.info(f"Skipping folder (prefix match): {foldername}")
            continue
        
        # Iterate through each file in the current folder
        for file in filenames:
            # Check if the file has the '.raw' extension
            if file.endswith('.raw'):
                # Check if the 9th and 10th characters (file[9:11]) are 'QC'
                if file[9:11] == 'QC':
                    # Create the full file path
                    file_path = os.path.join(foldername, file)
                    try:
                        # Delete the file
                        os.remove(file_path)
                        logging.info(f"Deleted file: {file_path}")
                    except Exception as e:
                        logging.error(f"Error deleting file: {file_path}, Error: {e}")

# Set the directory path you want to search in
directory_path = '/path/to/your/target_directory'

# Call the function to delete 'QC' raw files, excluding the most recent folder and a specific folder prefix
delete_qc_raw_files(directory_path, exclude_most_recent=True, exclude_folder_prefix='202407')
